In [ ]:
# Importing packages

import pandas as pd
import numpy as np
from scipy import stats
import scipy.stats as st
from statsmodels.stats.multicomp import pairwise_tukeyhsd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Combining all worksheets into a dataframe

df = pd.concat(pd.read_excel('/content/drive/MyDrive/Data Science/Agoda/Case_Study_Urgency_Message_Data.xlsx', sheet_name=None), ignore_index=True)

In [ ]:
# Calculating number of days stay

df['days_stay'] = (df['checkout_date'] - df['checkin_date']) / np.timedelta64(1, 'D')

In [ ]:
# Calculating number of days between booking and checking in

df['days_between_booking_checking'] = (df['checkin_date'] - df['booking_date'])/ np.timedelta64(1, 'D')
df['Days_Booked_Early'] = pd.cut(df['days_between_booking_checking'], 5)

In [ ]:
# Calculating price tiers

df['Price_Tier'] = pd.cut(df['ADR_USD'], 5, labels=["budget", "low", "medium", "high", "luxury"])

In [ ]:
df.head()

,#,ADR_USD,hotel_id,city_id,star_rating,accommodation_type_name,chain_hotel,booking_date,checkin_date,checkout_date,days_stay,days_between_booking_checking,Days_Booked_Early,Price_Tier
0,1,71.06,297388,9395,2.5,Hotel,non-chain,2016-08-02,2016-10-01,2016-10-02,1.0,60.0,"(47.8, 60.0]",budget
1,2,76.56,298322,9395,3.0,Hotel,non-chain,2016-08-02,2016-10-01,2016-10-02,1.0,60.0,"(47.8, 60.0]",budget
2,3,153.88,2313076,9395,5.0,Hotel,chain,2016-08-02,2016-10-01,2016-10-02,1.0,60.0,"(47.8, 60.0]",budget
3,4,126.60,2240838,9395,3.5,Hotel,non-chain,2016-08-04,2016-10-02,2016-10-03,1.0,59.0,"(47.8, 60.0]",budget
4,5,115.08,2240838,9395,3.5,Hotel,non-chain,2016-08-04,2016-10-02,2016-10-03,1.0,59.0,"(47.8, 60.0]",budget


In [ ]:
df = df.drop(columns=["#"])

In [ ]:
len(df)

49064

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49064 entries, 0 to 49063
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   ADR_USD                        49064 non-null  float64       
 1   hotel_id                       49064 non-null  int64         
 2   city_id                        49064 non-null  int64         
 3   star_rating                    49064 non-null  float64       
 4   accommodation_type_name        49064 non-null  object        
 5   chain_hotel                    49064 non-null  object        
 6   booking_date                   49064 non-null  datetime64[ns]
 7   checkin_date                   49064 non-null  datetime64[ns]
 8   checkout_date                  49064 non-null  datetime64[ns]
 9   days_stay                      49064 non-null  float64       
 10  days_between_booking_checking  49064 non-null  float64       
 11  Days_Booked_Ear

In [ ]:
df.isna().sum()

ADR_USD                          0
hotel_id                         0
city_id                          0
star_rating                      0
accommodation_type_name          0
chain_hotel                      0
booking_date                     0
checkin_date                     0
checkout_date                    0
days_stay                        0
days_between_booking_checking    0
Days_Booked_Early                0
Price_Tier                       0
dtype: int64

In [ ]:
for col in df.columns:
    print(col, '\n', df[col].value_counts(normalize=True).head(), '\n\n' )

ADR_USD 
 36.0    0.001203
38.0    0.001039
52.0    0.000999
50.0    0.000897
40.0    0.000815
Name: ADR_USD, dtype: float64 


hotel_id 
 197996     0.054602
21272      0.026781
219762     0.026170
16146      0.023255
1545890    0.021156
Name: hotel_id, dtype: float64 


city_id 
 9395     0.455854
16808    0.206913
5085     0.138533
17193    0.100522
8584     0.098178
Name: city_id, dtype: float64 


star_rating 
 3.0    0.301626
4.0    0.295288
3.5    0.128302
5.0    0.083157
1.0    0.069236
Name: star_rating, dtype: float64 


accommodation_type_name 
 Hotel                            0.722342
Resort                           0.071845
Serviced Apartment               0.057007
Hostel                           0.056538
Guest House / Bed & Breakfast    0.056212
Name: accommodation_type_name, dtype: float64 


chain_hotel 
 non-chain    0.73021
chain        0.26979
Name: chain_hotel, dtype: float64 


booking_date 
 2016-10-06    0.012698
2016-10-05    0.012616
2016-11-07    0.012555
2

In [ ]:
for col in df.columns:
    print(col, '\n', len(df[col].unique()))

ADR_USD 
 20210
hotel_id 
 880
city_id 
 5
star_rating 
 10
accommodation_type_name 
 16
chain_hotel 
 2
booking_date 
 151
checkin_date 
 92
checkout_date 
 94
days_stay 
 3
days_between_booking_checking 
 62
Days_Booked_Early 
 5
Price_Tier 
 5


In [ ]:
df.describe()

,ADR_USD,hotel_id,city_id,star_rating,days_stay,days_between_booking_checking
count,49064.000000,4.906400e+04,49064.000000,49064.000000,49064.000000,49064.000000
mean,148.089095,7.921256e+05,11036.016937,3.328887,1.676280,14.529472
std,135.338544,7.559906e+05,4174.737590,0.994740,0.772219,15.680648
min,4.260000,7.700000e+02,5085.000000,0.000000,1.000000,-1.000000
25%,62.555000,1.979960e+05,9395.000000,3.000000,1.000000,2.000000
50%,113.660000,5.260860e+05,9395.000000,3.500000,1.000000,8.000000
75%,192.220000,1.249476e+06,16808.000000,4.000000,2.000000,23.000000
max,3156.860000,5.211366e+06,17193.000000,5.000000,3.000000,60.000000


In [ ]:
# Writing a helper function to calculate the Welch's t-value

def welch_t(a, b):

    numerator = np.mean(a) - np.mean(b)
    
    denominator = np.sqrt((np.var(a, ddof=1)/len(a)) + (np.var(b, ddof=1)/len(b)))
    
    return np.abs(numerator/denominator)

# Writing a helper function to calculate the Welch's df

def welch_df(a, b):
    
    s1 = np.var(a, ddof=1) 
    s2 = np.var(b, ddof=1)
    n1 = len(a)
    n2 = len(b)
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

# Writing a function to calculate the Welch's p-value

def p_value(a, b, two_sided=False):

    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-st.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [ ]:
# creating two data sets based on chain vs non-chain

chain = df[df['chain_hotel']=='chain']['ADR_USD']
non_chain = df[df['chain_hotel']=='non-chain']['ADR_USD']

In [ ]:
# Performing Welch's t-test between chain vs non-chain

print("Median Values: \ts1: {} \ts2: {}".format(round(np.median(chain), 2), round(np.median(non_chain), 2)))
print("Mean Values: \ts1: {} \ts2: {}".format(round(np.mean(chain), 2), round(np.mean(non_chain), 2)))
print('Sample sizes: \ts1: {} \ts2: {}'.format(len(chain), len(non_chain)))
print("Welch's t-test p-value:", p_value(chain, non_chain, two_sided=True))

Median Values: 	s1: 161.0 	s2: 91.64
Mean Values: 	s1: 200.18 	s2: 128.84
Sample sizes: 	s1: 13237 	s2: 35827
Welch's t-test p-value: 0.0


In [ ]:
# Tukey testing between days booked early

print(pairwise_tukeyhsd(df['ADR_USD'], df['Days_Booked_Early'], 0.05))

       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1        group2    meandiff p-adj   lower   upper  reject
------------------------------------------------------------------
(-1.061, 11.2] (11.2, 23.4]  23.4011  0.001 18.9515 27.8506   True
(-1.061, 11.2] (23.4, 35.6]  28.0666  0.001 22.6578 33.4754   True
(-1.061, 11.2] (35.6, 47.8]  31.6749  0.001 25.2975 38.0524   True
(-1.061, 11.2] (47.8, 60.0]  42.4082  0.001 35.0928 49.7236   True
  (11.2, 23.4] (23.4, 35.6]   4.6655 0.2533 -1.6155 10.9466  False
  (11.2, 23.4] (35.6, 47.8]   8.2739 0.0135  1.1417 15.4061   True
  (11.2, 23.4] (47.8, 60.0]  19.0071  0.001 11.0252 26.9891   True
  (23.4, 35.6] (35.6, 47.8]   3.6083 0.6861 -4.1586 11.3753  False
  (23.4, 35.6] (47.8, 60.0]  14.3416  0.001  5.7877 22.8954   True
  (35.6, 47.8] (47.8, 60.0]  10.7332 0.0127  1.5363 19.9302   True
------------------------------------------------------------------


In [ ]:
# Tukey testing between days_stay

print(pairwise_tukeyhsd(df['ADR_USD'], df['days_stay'], 0.05))

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower   upper  reject
---------------------------------------------------
   1.0    2.0  21.4065 0.001 18.1211  24.692   True
   1.0    3.0  22.1422 0.001 18.2983 25.9861   True
   2.0    3.0   0.7357   0.9 -3.4615  4.9328  False
---------------------------------------------------


In [ ]:
# Tukey testing between price tiers

print(pairwise_tukeyhsd(df['ADR_USD'], df['Price_Tier'], 0.05))

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
group1 group2  meandiff  p-adj   lower      upper    reject
-----------------------------------------------------------
budget   high  2131.0939 0.001  2056.0673  2206.1206   True
budget    low    684.908 0.001   671.2652   698.5508   True
budget luxury  2638.9558 0.001  2476.8981  2801.0135   True
budget medium  1333.4353 0.001  1295.5665  1371.3041   True
  high    low -1446.1859 0.001 -1522.4216 -1369.9502   True
  high luxury   507.8619 0.001   329.2885   686.4353   True
  high medium  -797.6586 0.001  -881.6812   -713.636   True
   low luxury  1954.0478 0.001  1791.4268  2116.6688   True
   low medium   648.5273 0.001   608.3163   688.7383   True
luxury medium -1305.5205 0.001 -1471.9341 -1139.1068   True
-----------------------------------------------------------


In [ ]:
# Tukey testing between accomodation types

print(pairwise_tukeyhsd(df['ADR_USD'], df['accommodation_type_name'], 0.05))

                          Multiple Comparison of Means - Tukey HSD, FWER=0.05                          
            group1                        group2             meandiff p-adj    lower     upper   reject
-------------------------------------------------------------------------------------------------------
                    Apartment                      Bungalow  -124.444  0.001 -172.0174  -76.8705   True
                    Apartment                 Capsule Hotel -134.9972  0.001 -167.9937 -102.0007   True
                    Apartment Guest House / Bed & Breakfast -110.5466  0.001 -140.8222   -80.271   True
                    Apartment   Holiday Park / Caravan Park  -98.1175    0.9  -358.969  162.7339  False
                    Apartment                          Home    -1.839    0.9 -125.3036  121.6256  False
                    Apartment                        Hostel -107.5284  0.001  -137.797  -77.2598   True
                    Apartment                         Hotel  -38

In [ ]:
# Tukey testing between cities

print(pairwise_tukeyhsd(df['ADR_USD'], df['city_id'], 0.05))

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower     upper   reject
---------------------------------------------------------
  5085   8584  -82.7709  0.001  -89.1344  -76.4075   True
  5085   9395  -124.331  0.001 -129.0106 -119.6513   True
  5085  16808   -5.9912 0.0173  -11.2865    -0.696   True
  5085  17193 -106.5209  0.001 -112.8408  -100.201   True
  8584   9395    -41.56  0.001  -46.9268  -36.1933   True
  8584  16808   76.7797  0.001   70.8684   82.6909   True
  8584  17193    -23.75  0.001  -30.5943  -16.9057   True
  9395  16808  118.3397  0.001  114.2964  122.3831   True
  9395  17193   17.8101  0.001    12.495   23.1251   True
 16808  17193 -100.5297  0.001  -106.394  -94.6653   True
---------------------------------------------------------


In [ ]:
# Tukey testing between star ratings

print(pairwise_tukeyhsd(df['ADR_USD'], df['star_rating'], 0.05))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
   0.0    1.0  41.0679  0.001  10.9657  71.1702   True
   0.0    1.5  18.5417 0.7665 -15.8715  52.9549  False
   0.0    2.0   4.6253    0.9 -25.8015  35.0521  False
   0.0    2.5  17.8514 0.6816 -12.7979  48.5007  False
   0.0    3.0  83.2917  0.001  53.7536 112.8298   True
   0.0    3.5  43.8796  0.001  14.1132  73.6461   True
   0.0    4.0 142.2683  0.001 112.7265   171.81   True
   0.0    4.5 117.0062  0.001  85.4256 148.5869   True
   0.0    5.0 231.6637  0.001 201.6831 261.6443   True
   1.0    1.5 -22.5262 0.0074 -41.6433  -3.4092   True
   1.0    2.0 -36.4427  0.001  -46.786 -26.0993   True
   1.0    2.5 -23.2165  0.001  -34.197 -12.2359   True
   1.0    3.0  42.2238  0.001  34.8962  49.5514   True
   1.0    3.5   2.8117    0.9   -5.388  11.0114  False
   1.0    4.0 101.2003  0.001  93.8581 108.5426   True
   1.0    

In [ ]:
df.to_excel("/content/drive/MyDrive/Data Science/Agoda/agoda_output.xlsx")  